In [4]:
import pandas as pd 
import bibtexparser as bbp 
import operations as op
import yaml

ACM_DF = pd.DataFrame(bbp.load(open("../data/ACM.bib", encoding="utf8")).entries)
IEE_DF = pd.DataFrame(bbp.load(open("../data/IEEE.bib", encoding="utf8")).entries)
SCIENCE_DIRECT_DF = pd.DataFrame(bbp.load(open("../data/ScienceDirect.bib", encoding="utf8")).entries)

OUTPUT_FOLDER = './output'

CONFIG = yaml.load(open("./config.yaml"), yaml.loader.SafeLoader)


In [5]:


if CONFIG.get("export_format") not in ["json", "csv", "yaml", "xml"]:
    raise Exception("O formato de exportação especificado no arquivo de configuração .yaml não foi reconhecido (json, csv, yaml, xml)")

EXPORT_TYPE = CONFIG.get("export_format")

acm_df = (ACM_DF
          .pipe(op.map, ["author", "title", "keywords", "abstract", "year", "ENTRYTYPE", "doi"])
          .pipe(op.rename_cols, {
                "ENTRYTYPE": "type_publication"
          }))

iee_df = (IEE_DF
          .pipe(op.map, ["author", "title", "keywords", "abstract", "year", "ENTRYTYPE", "doi"])
          .pipe(op.rename_cols, {
                "ENTRYTYPE": "type_publication"
          }))

science_direct_df = (SCIENCE_DIRECT_DF
          .pipe(op.map, ["author", "title", "keywords", "abstract", "year", "ENTRYTYPE", "doi"])
          .pipe(op.rename_cols, {
                "ENTRYTYPE": "type_publication"
          }))

if EXPORT_TYPE == "json":
    acm_df.to_json(f"{OUTPUT_FOLDER}/acm.json", orient="records")
    iee_df.to_json(f"{OUTPUT_FOLDER}/iee.json", orient="records")
    science_direct_df.to_json(f"{OUTPUT_FOLDER}/science_direct.json", orient="records")

elif EXPORT_TYPE == "csv":
    acm_df.to_csv(f"{OUTPUT_FOLDER}/acm.csv", index=False, sep="|")
    iee_df.to_csv(f"{OUTPUT_FOLDER}/iee.csv", index=False, sep="|")
    science_direct_df.to_csv(f"{OUTPUT_FOLDER}/science_direct.csv", index=False, sep="|")

elif EXPORT_TYPE == "yaml":
    def to_yaml(df: pd.DataFrame, file_name: str):
        open(f"{OUTPUT_FOLDER}/{file_name}", 'w').write(
            yaml.dump(df.to_dict(orient="records")))

    to_yaml(acm_df, "acm.yaml")
    to_yaml(iee_df, "iee.yaml")
    to_yaml(science_direct_df, "science_direct.yaml")

elif EXPORT_TYPE == "xml":
    acm_df.to_xml(f"{OUTPUT_FOLDER}/acm.xml", index=False)
    iee_df.to_xml(f"{OUTPUT_FOLDER}/iee.xml", index=False)
    science_direct_df.to_xml(f"{OUTPUT_FOLDER}/science_direct.xml", index=False)
    
